# Infeasibility UQ

Take the generic DEMO solution, turned into an input file. Remove f-values at iteration vars, and replace their equality constraints with inequalities. Run PROCESS once-through with uncertain inputs, and the QoI as the value of constraints, i.e. the infeasibility.

Dask is used to parallelise the evaluations for a SLURM cluster.


In [1]:
import easyvvuq as uq
import chaospy as cp
from pathlib import Path
from dask.distributed import Client
from dask_jobqueue import SLURMCluster
import shutil
import os

## Campaign to capture feasibility

Using the epistemic uncertain inputs for the entire code, capture the distribution of constraint residuals.

To start with, make just 4 inputs uncertain.


In [2]:
# Init cluster (describes a single node, or less if need less than that per worker)
# cluster = SLURMCluster(
#     cores=56,
#     processes=4,  # check docs
#     memory="192GB",
#     account="UKAEA-AP002-CPU",
#     walltime="01:00:00",
#     queue="cclake",
# )

# Need less than a full node per worker
cluster = SLURMCluster(
    cores=1,
    processes=1,
    memory="4GB",
    account="UKAEA-AP002-CPU",
    walltime="00:45:00",
    queue="cclake",
)
cluster.scale(10)  # 4 workers
# print(cluster.job_script())

# Connect Dask client to remote cluster
client = Client(cluster)
# Code from now on submitted to batch queue

# Define campaign
WORK_DIR = "campaigns"
Path("campaigns").mkdir(exist_ok=True)
campaign = uq.Campaign(name="example_cluster", work_dir=WORK_DIR)

# Define parameter space
# Uncertainties from Alex"s SA paper

params = {
    "coreradius": {
        "type": "float",
        "min": 0.6,
        "max": 0.9,
        "default": 0.7
    },
    "ralpne": {
        "type": "float",
        "min": 0.072,
        "max": 0.0761,
        "default": 0.74
    }, 
    "psepbqarmax": {
        "type": "float",
        "min": 7.359,
        "max": 11.04,
        "default": 9.0
    },
    "tbrnmn": {
        "type": "float",
        "min": 6954.0,
        "max": 12722.0,
        "default": 10000.0 
    },
    "etaech": {
        "type": "float",
        "min": 0.32,
        "max": 0.49,
        "default":0.40
    }, 
    "pinjalw": {
        "type": "float",
        "min": 40.0,
        "max": 60.0,
        "default": 50.0
    }, 
    "triang": {
        "type": "float",
        "min": 0.4,
        "max": 0.6,
        "default":0.5
    }, 
    "alstroh": {
        "type": "float",
        "min": 528000000.0,
        "max": 792000000.0,
        "default":628000000.0 
    }, 
    "sig_tf_case_max": {
        "type": "float",
        "min": 464000000.,
        "max": 696000000.0,
        "default": 596000000.0
    }, 
    "walalw": {
        "type": "float",
        "min": 6.4,
        "max": 9.6,
        "default": 7.4
    }, 
    "sig_tf_wp_max": {
        "type": "float",
        "min": 528000000.0,
        "max": 792000000.0,
        "default": 692000000.0
    }, 
    "aspect": {
        "type": "float",
        "min": 2.785,
        "max": 2.935,
        "default": 2.82
    }, 
    "vary_param": {
        "type": "string",
        "default": "",
    },  # param being changed: used for analysis only
}

# QoIs
# Violated constraint residuals
qois = [
    "coheof","bore","powfmw","ifail","sqsumsq","rmajor"
]


# Create encoder and decoder
encoder1 = uq.encoders.GenericEncoder(
    template_fname="clean_demo_2023_IN.template", target_filename="IN.DAT"
)
encoder2 = uq.encoders.GenericEncoder(
    template_fname="MFILE.DAT.json", target_filename="MFILE.DAT.json"
)
decoder = uq.decoders.JSONDecoder(target_filename="MFILE.DAT.json", output_columns=qois)
my_multiencoder = uq.encoders.MultiEncoder(encoder1, encoder2)


cmd = "process -i IN.DAT --mfilejson --mfile MFILE.DAT"
actions = uq.actions.local_execute(my_multiencoder, cmd, decoder)

# Add the app
campaign.add_app(name="feasibility", params=params, actions=actions)

# Create PCE sampler, 4 uncertainties
# vary = {
#     "aspect": cp.Uniform(2.785,2.935),
#     "triang": cp.Uniform(0.4, 0.6),
#     "psepbqarmax": cp.Uniform(8.7, 9.7),
#     "hfact": cp.Uniform(1.0, 1.2),
# }
vary={"coreradius": cp.Uniform(0.6,0.9),
      "ralpne": cp.Uniform(0.072,0.0761),
      "psepbqarmax": cp.Uniform(7.359,11.04),
      "tbrnmn": cp.Uniform(6954.0,12722.0),
      "etaech": cp.Uniform(0.32,0.49),
      "pinjalw": cp.Uniform(40.0,60.0),
      "triang": cp.Uniform(0.4,0.6),
      "alstroh": cp.Uniform(528000000.0,792000000.0),
      "sig_tf_case_max": cp.Uniform(464000000.0,696000000.0),
      "walalw": cp.Uniform(6.4,9.6),
      "sig_tf_wp_max": cp.Uniform(528000000.0,792000000.0),
      "aspect": cp.Uniform(2.785,2.935),}

pce_sampler = uq.sampling.MCSampler(vary=vary,n_mc_samples=1)

# Add pce_sampler to campaign
campaign.set_sampler(pce_sampler)

# Draw samples, execute and collate
campaign.execute(pool=client).collate(progress_bar=True)
samples = campaign.get_collation_result()
samples


  0%|          | 0/30 [00:00<?, ?it/s]


100%|██████████| 30/30 [00:00<00:00, 1136.69it/s]

,run_id,iteration,aspect,triang,psepbqarmax,hfact,fdene,coreradius,fimp_2,fimp_14,...,etaiso,boundl_18,pinjalw,alstroh,sig_tf_wp_max,boundu_2,vary_param,coheof,bore,powfmw
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,1,0,3.140974,0.447581,8.755863,1.070616,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2574.1
1,2,0,3.100607,0.447581,8.755863,1.070616,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2635.3
2,3,0,3.140974,0.484047,8.755863,1.070616,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2556.6
3,4,0,3.140974,0.447581,8.880050,1.070616,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2574.1
4,5,0,3.140974,0.447581,8.755863,1.008237,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2574.1
5,6,0,3.100607,0.484047,8.880050,1.008237,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2617.2
6,7,0,3.031227,0.429748,9.425251,1.073415,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2754.2
7,8,0,3.069165,0.429748,9.425251,1.073415,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2693.2
8,9,0,3.031227,0.540330,9.425251,1.073415,1.2,0.75,0.1,0.00001,...,0.9,3.25,61.0,660000000.0,640000000.0,12.0,,9102000.0,2.8236,2695.7
